# Applications library + krisszi-core

matplotlib  3.10.3 -> 3.9.4

device_setup 부분 현재 config/ 안에 laboneq.serializer 이용하여 device_setup.json & instruments.json 형태로 저장 (backend.setup_builder.add_instruments & backend.setup_builder.save_device_setup)  

In [1]:
%load_ext autoreload
%autoreload 2

from krisszi_core.config.qbackend import QuantumBackend
from krisszi_core.contrib.chip_template.GINQO_1a import qpu_chip
import numpy as np
from krisszi_core.handler.result_handler import ResultsHandler
from krisszi_core.dsl.options import RunOptions, SaveOptions, AnalysisOptions
import matplotlib.pyplot as plt
from laboneq.simple import AcquisitionType, AveragingMode
import logging



%matplotlib inline
# %matplotlib qt5

from laboneq.contrib.example_helpers.plotting.plot_helpers import *
from laboneq.simple import *


In [2]:
qpu_chip.active_list = [3]
parent_path = r"C:\Users\KRISS\Documents\KAIST"
# backend = QuantumBackend(
#     "2QPU_BF7",
#     qpu_chip,
#     parent_path,
#     host="10.172.11.217",
#     port = "8004",
# )

backend = QuantumBackend(
    "2QPU_BF7",
    qpu_chip,
    parent_path,
    host="10.172.11.217",
    port = "8004"
)
backend.setup_builder.clear()
backend.setup_builder.add_instrument(
    3, "SHFQC", "DEV12576", 3, dev_option="/PLUS/QC6CH"
)
# backend.setup_builder.add_instrument(
#     1, "SHFQC", "DEV12566", 1, dev_option="/PLUS/QC6CH"
# )
backend.setup_builder.save_device_setup() #device_setup 없으면 알아서 build 이후 json으로 save
backend.session.disconnect()
backend.connect(False, log_level=logging.INFO)


backend.qpu.to_dataframe(level=1)

[2025.07.29 19:34:47.725] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is c:\Users\KRISS\Documents\KAIST\krisszi-core\src\krisszi_core\contrib\jsahn\laboneq_output\log
[2025.07.29 19:34:47.902] INFO    VERSION: laboneq 2.56.0
[2025.07.29 19:34:47.903] INFO    Connecting to data server at 10.172.11.217:8004
[2025.07.29 19:34:47.912] INFO    Connected to Zurich Instruments LabOne Data Server version 25.04.1.17 at 10.172.11.217:8004
[2025.07.29 19:34:47.940] INFO    Configuring the device setup
[2025.07.29 19:34:47.949] INFO    The device setup is configured


,fr [GHz],f01 [GHz],Δ [GHz],f12 [GHz],α [MHz],χ [MHz],κ [kHz],g [MHz],T2* [μs],ε10 [%],ε01 [%],F_RO [%],T1 [μs],T2 [μs]
15,7.07649,4.82658,2.24991,5.27342,-446.8,0.00,987.0,3.2,23.3,23.02,23.17,76.91,NaN,NaN
16,6.99757,4.80218,2.19538,4.51255,289.6,0.01,908.0,4.9,9.7,25.66,39.75,67.29,29.2,NaN
17,6.92253,4.71608,2.20645,4.43792,278.2,0.82,1126.0,42.5,17.0,2.95,6.54,95.25,31.4,32.4
18,6.84328,4.66525,2.17803,4.37635,288.9,0.81,986.0,41.9,44.8,3.39,5.25,95.68,47.7,65.4
19,6.77385,4.77902,1.99484,4.48273,296.3,0.01,536.0,3.7,19.9,10.13,8.01,90.93,28.0,47.4


실험할때 session = Session(setup) 이런식으로 calibration 넣는 부분

In [ ]:
#backend.session
setup = backend.setup_builder.device_setup 
print(setup.check_no_rf_multiplexing)
from laboneq.serializers import load
setup2 = load('c:/Users/KRISS/Documents/KAIST/qc_backend/2QPU_BF7/config/device_setup.json')

In [ ]:
setup.instruments

In [ ]:
def get_physical_signal_name(quid, signal_name):
    logical_signal = setup.logical_signal_groups[quid].logical_signals[signal_name]
    return logical_signal.physical_channel.uid


qubit_signals = {
    quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
}
connections = {
    quid: {sig_name: get_physical_signal_name(quid, sig_name) for sig_name in signals}
    for quid, signals in qubit_signals.items()
}

from pprint import pprint

pprint(connections)  # noqa: T203

In [ ]:
qubit_signals2 = {
    quid: list(lsg.logical_signals) for quid, lsg in setup2.logical_signal_groups.items()
}
connections2 = {
    quid: {sig_name: get_physical_signal_name(quid, sig_name) for sig_name in signals}
    for quid, signals in qubit_signals2.items()
}

pprint(connections)  # noqa: T203

In [ ]:
setup.logical_signal_groups['q17']

Do we need another LogicalSignal just for cr drive? can't it be just 'drive'?



In [ ]:
from pathlib import Path
Path.cwd()

In [ ]:
import numpy as np
from laboneq.core.exceptions import LabOneQException
from laboneq.simple import *
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from laboneq.dsl.quantum.qpu import QuantumElement

from krisszi_core.contrib.jsahn.operations import TransmonOperations
from krisszi_core.contrib.jsahn.helper import load_qubit_parameters





loaded_qparams = load_qubit_parameters(filename="latest") #dict with quid as key and TransmonQubit object as value
qubits = list(loaded_qparams.values())

quantum_operations = TransmonOperations()
qpu = QPU(quantum_elements=qubits, quantum_operations=quantum_operations)


# Create a demonstration QuantumPlatform for a transmon QPU:

quantum_platform = QuantumPlatform(setup=setup, qpu=qpu) 

In [ ]:
quantum_platform.setup

In [ ]:
from laboneq_applications.experiments import ramsey 